In [334]:
import numpy as np
import pandas as pd
import pyodbc
import matplotlib.pyplot as plt

In [335]:
# Solvers 
from scipy.optimize import root as root
from scipy.optimize import brentq as brentq

Parametres

In [336]:
session_date = '2020-06-29'
# If no session id, then take published session
session_id   = None

risk_ids = "('IND_CDP_CAISSE', 'FUND_IND_MSCI_ACWI_EX_CANADA', 'IND_SP_TSX_CAP', 'FUND_IND_DEX_UNIV')"

# Indice CAISSE: IND_CDP_CAISSE
indice_caisse_id = 'IND_CDP_CAISSE';

# Indices de base pour MB     
indice_pour_mb_1_id = 'FUND_IND_MSCI_ACWI_EX_CANADA';
indice_pour_mb_1_percent = 0.80;
indice_pour_mb_2_id = 'IND_SP_TSX_CAP';
indice_pour_mb_2_percent = 0.20;

# Indices de base pour RF      
indice_pour_rf_1_id = 'FUND_IND_DEX_UNIV';
indice_pour_mb_rf_percent = 1.0;

# Calcul de la VaR
percent = 95
metho = 'RobustHD'
timeFactor = 13

In [337]:
print(pyodbc)
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=CECGENSQL4\MECZAPRSK01;'
                      'Database=rsq_model;'
                      'Trusted_Connection=yes;')
print(con)

<module 'pyodbc' from 'C:\\ProgramData\\Anaconda3\\envs\\DLENV\\lib\\site-packages\\pyodbc.cp37-win_amd64.pyd'>


conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=server_name;'
                      'Database=db_name;'
                      'Trusted_Connection=yes;')


In [338]:
query_template = """
SELECT   
	  IndiceIDRisque
	  , [SessionDate]
      ,[SessionID]
      ,[DynamicIdentifier]
      ,[CaracteristiqueScenarioLibelle]
      ,[IndicateurDonneePubliee]
      ,[FrequenceSessionCode]
      ,[TypeScenario]
      ,[HorizonCalcul]
      ,[NombreObservation]
      ,[ValeurMarchande]
      , A.Date as ScenarioDate
	  , A.Pnl  as Pnl
	  , A.Pnl/ValeurMarchande as Pnl_percent
  FROM  [MesureRisqueMarche].[V_PnLRMIndiceVecteur] V
  cross apply [dbo].[PivotDatePnl] ([VecteurScenariodate], vecteurIndicePnl, ';') A
  where SessionDate = '{}'
  and {}
  and IndiceIDRisque in {}
  
"""

if session_id is None:
    query_session_selection = 'IndicateurDonneePubliee = 1'
else:
    query_session_selection = 'sessionId = {}'.format(session_id)

query = query_template.format(session_date, query_session_selection, risk_ids)

print(query)


SELECT   
	  IndiceIDRisque
	  , [SessionDate]
      ,[SessionID]
      ,[DynamicIdentifier]
      ,[CaracteristiqueScenarioLibelle]
      ,[IndicateurDonneePubliee]
      ,[FrequenceSessionCode]
      ,[TypeScenario]
      ,[HorizonCalcul]
      ,[NombreObservation]
      ,[ValeurMarchande]
      , A.Date as ScenarioDate
	  , A.Pnl  as Pnl
	  , A.Pnl/ValeurMarchande as Pnl_percent
  FROM  [MesureRisqueMarche].[V_PnLRMIndiceVecteur] V
  cross apply [dbo].[PivotDatePnl] ([VecteurScenariodate], vecteurIndicePnl, ';') A
  where SessionDate = '2020-06-29'
  and IndicateurDonneePubliee = 1
  and IndiceIDRisque in ('IND_CDP_CAISSE', 'FUND_IND_MSCI_ACWI_EX_CANADA', 'IND_SP_TSX_CAP', 'FUND_IND_DEX_UNIV')
  



In [339]:
# Load data
data = pd.read_sql(query, con)

In [340]:
data.shape

(15040, 14)

In [341]:
data.NombreObservation.unique()[0]

3760

In [342]:
data.head(10)

,IndiceIDRisque,SessionDate,SessionID,DynamicIdentifier,CaracteristiqueScenarioLibelle,IndicateurDonneePubliee,FrequenceSessionCode,TypeScenario,HorizonCalcul,NombreObservation,ValeurMarchande,ScenarioDate,Pnl,Pnl_percent
0,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-29,1.897176e+09,0.013864
1,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-26,1.684572e+09,0.012310
2,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-25,2.064747e+09,0.015088
3,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-24,1.607615e+09,0.011748
4,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-23,1.788941e+09,0.013073
5,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-22,1.388963e+09,0.010150
6,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-19,1.629813e+09,0.011910
7,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-18,1.918419e+09,0.014019
8,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-17,1.749468e+09,0.012784
9,FUND_IND_DEX_UNIV,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.368446e+11,2020-06-16,1.977796e+09,0.014453


In [343]:
data.IndiceIDRisque

0        FUND_IND_DEX_UNIV
1        FUND_IND_DEX_UNIV
2        FUND_IND_DEX_UNIV
3        FUND_IND_DEX_UNIV
4        FUND_IND_DEX_UNIV
               ...        
15035       IND_SP_TSX_CAP
15036       IND_SP_TSX_CAP
15037       IND_SP_TSX_CAP
15038       IND_SP_TSX_CAP
15039       IND_SP_TSX_CAP
Name: IndiceIDRisque, Length: 15040, dtype: object

In [344]:
is_indice_caisse = data.IndiceIDRisque == indice_caisse_id
data[is_indice_caisse] 

,IndiceIDRisque,SessionDate,SessionID,DynamicIdentifier,CaracteristiqueScenarioLibelle,IndicateurDonneePubliee,FrequenceSessionCode,TypeScenario,HorizonCalcul,NombreObservation,ValeurMarchande,ScenarioDate,Pnl,Pnl_percent
7520,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2020-06-29,0.011419,0.011419
7521,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2020-06-26,0.007566,0.007566
7522,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2020-06-25,0.016778,0.016778
7523,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2020-06-24,0.012449,0.012449
7524,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2020-06-23,0.027886,0.027886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11275,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2006-02-06,-0.001686,-0.001686
11276,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2006-02-03,-0.001046,-0.001046
11277,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2006-02-02,0.006615,0.006615
11278,IND_CDP_CAISSE,2020-06-29,2565,CDPQ_20200629_Daily_1,Historique_4W_20060131,True,D,historique,20,3760,1.0,2006-02-01,0.013039,0.013039


In [345]:
v_caisse = data[data.IndiceIDRisque == indice_caisse_id].Pnl_percent
v_caisse

7520     0.011419
7521     0.007566
7522     0.016778
7523     0.012449
7524     0.027886
           ...   
11275   -0.001686
11276   -0.001046
11277    0.006615
11278    0.013039
11279    0.019076
Name: Pnl_percent, Length: 3760, dtype: float64

In [346]:
v_mb_1 = data[data.IndiceIDRisque == indice_pour_mb_1_id].Pnl_percent
v_mb_1

3760    0.022768
3761    0.012649
3762    0.030139
3763    0.020084
3764    0.047477
          ...   
7515   -0.028136
7516   -0.019802
7517    0.000040
7518    0.020207
7519    0.028666
Name: Pnl_percent, Length: 3760, dtype: float64

In [347]:
v_mb_2 = data[data.IndiceIDRisque == indice_pour_mb_2_id].Pnl_percent
v_mb_2

11280    0.017521
11281    0.006101
11282    0.018527
11283    0.008114
11284    0.033386
           ...   
15035    0.039880
15036    0.028607
15037    0.036424
15038    0.034504
15039    0.033853
Name: Pnl_percent, Length: 3760, dtype: float64

In [348]:
v_rf_1 = data[data.IndiceIDRisque == indice_pour_rf_1_id].Pnl_percent
v_rf_1

0       0.013864
1       0.012310
2       0.015088
3       0.011748
4       0.013073
          ...   
3755   -0.009722
3756   -0.009237
3757   -0.014050
3758   -0.017416
3759   -0.012569
Name: Pnl_percent, Length: 3760, dtype: float64

In [349]:
v_mb = indice_pour_mb_1_percent * np.array(v_mb_1) + indice_pour_mb_2_percent * np.array(v_mb_2)
v_mb

array([0.02171887, 0.01133982, 0.02781667, ..., 0.00731655, 0.02306682,
       0.029703  ])

In [350]:
v_rf = indice_pour_mb_rf_percent * np.array(v_rf_1)  
v_rf

array([ 0.01386372,  0.01231011,  0.01508826, ..., -0.0140504 ,
       -0.01741596, -0.01256941])

In [351]:
# Get Poids HD
def getHDWeights(percent, metho, nb_observation):
    hd_query = """
    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    """ 
    print(hd_query)
    hd = pd.read_sql(hd_query.format(metho, percent, nb_observation), con)
    hd.head()

    # Sort by Rang
    return hd.sort_values('Rang')

In [352]:
percent = 95
metho = 'RobustHD'
nb_observation = data.NombreObservation.unique()[0]
print(nb_observation)

hd = getHDWeights(percent, metho, nb_observation)
# hd['Poids']

3760

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    


In [353]:
from math import sqrt, pow

def calculVaR(percent, metho, nb_observation, timeFactor, positionAmount ,  v_pnl): 
# Fonction qui calcule la VaR des pnls considérés 
    # 1- Poids HD
    v_hdWeights = getHDWeights(percent, metho, nb_observation)
    #print(hdWeights.head())

    # 2- Sort Pnls
    #print(v_pnl)
    v_pnl_sorted = np.sort(v_pnl)
    #print(v_pnl_sorted)

    # 3 sort HD 
    v_hdWeights_sorted = v_hdWeights.sort_values('Rang')['Poids']

    # Multiply Pnl by HD
    print(v_pnl_sorted.shape, v_hdWeights_sorted.shape) 
    v_pnl_hd = v_pnl_sorted * v_hdWeights_sorted
    # print(v_pnl_hd.shape)

    # Average, Percentile
    average = np.average(v_pnl)
    sum = np.sum(v_pnl_hd)
    percentile = sum 
    # print(average, sum, percentile)

    # Annualized
    annualizedVaR = 1- pow(1 + (percentile - average) / positionAmount, sqrt(timeFactor))
    # return var 
    return annualizedVaR
 

In [354]:
nb_observation = data.NombreObservation.unique()[0]
positionAmount = 1 # Valeur marchande = 1 si %
print(nb_observation)


3760


In [355]:
# VaR Caisse
var_Caisse = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_caisse) 
print(var_Caisse)
var_target = var_Caisse

#print(v_caisse)


    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
0.13691006787276627


In [356]:
var_mb_1 = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_mb_1) 
print(var_mb_1)
 


    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
0.22708480854080748


In [357]:
var_mb_2 = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_mb_2) 
print(var_mb_2)
 


    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
0.20701090545262046


In [358]:
var_mb = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_mb) 
print(var_mb)
print(v_mb)


    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
0.2175687344852335
[0.02171887 0.01133982 0.02781667 ... 0.00731655 0.02306682 0.029703  ]


In [359]:
var_rf_1 = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_rf_1) 
print(var_rf_1)



    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
0.06983071382570982


In [360]:
var_rf = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount, v_rf) 
print(var_rf)
print(v_rf)


    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
0.06983071382570982
[ 0.01386372  0.01231011  0.01508826 ... -0.0140504  -0.01741596
 -0.01256941]


In [361]:
def getVectorPLS(x:float):
    pnl =  x * v_mb + (1-x) * v_rf
    print(pnl[:10])
    print(pnl.shape)
    return pnl  

In [362]:
def calculateVaRPLS(x:float):
    pnl =  getVectorPLS(x)
    var = calculVaR(percent, metho, nb_observation, timeFactor, positionAmount ,  pnl)
    return abs(var)  

In [363]:
# Fonction a optimiser
def toOptimize(x:float):
    var = calculateVaRPLS(x)
    return abs(var) - abs(var_target)

In [364]:
print(var_target)
 

0.13691006787276627


In [365]:
# Optimize using root

sol = root(toOptimize, 1)
sol

[0.02171887 0.01133982 0.02781667 0.01768976 0.04465854 0.0425582
 0.04354729 0.04740938 0.04232882 0.06090862]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
[0.02171887 0.01133982 0.02781667 0.01768976 0.04465854 0.0425582
 0.04354729 0.04740938 0.04232882 0.06090862]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
[0.02171887 0.01133982 0.02781667 0.01768976 0.04465854 0.0425582
 0.04354729 0.04740938 0.04232882 0.06090862]
(3760,)

    SELECT 
        -- [Metho

    fjac: array([[-1.]])
     fun: 0.0
 message: 'The solution converged.'
    nfev: 7
     qtf: array([2.44249065e-15])
       r: array([-0.19107831])
  status: 1
 success: True
       x: array([0.59566966])

Résultat 1: Méthode root

In [366]:
x1 = sol.x[0]
print(x1)

0.5956696556387304


In [367]:
toOptimize(x1)

[0.0185428  0.01173214 0.02267019 0.01528723 0.03188746 0.02945456
 0.03075535 0.03390862 0.03038309 0.04212515]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)


0.0

In [368]:
getVectorPLS(x1)

[0.0185428  0.01173214 0.02267019 0.01528723 0.03188746 0.02945456
 0.03075535 0.03390862 0.03038309 0.04212515]
(3760,)


array([ 0.0185428 ,  0.01173214,  0.02267019, ..., -0.00132276,
        0.0066984 ,  0.01261098])

In [369]:
calculateVaRPLS(x1)

[0.0185428  0.01173214 0.02267019 0.01528723 0.03188746 0.02945456
 0.03075535 0.03390862 0.03038309 0.04212515]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)


0.13691006787276627

Résultat 2: Méthode brentq

In [370]:
toOptimize(1)

[0.02171887 0.01133982 0.02781667 0.01768976 0.04465854 0.0425582
 0.04354729 0.04740938 0.04232882 0.06090862]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)


0.08065866661246723

In [371]:
#Optimize using brentq
x2 = brentq(toOptimize, 0, 1)

print(x2)

[0.01386372 0.01231011 0.01508826 0.01174775 0.01307279 0.01014993
 0.01190996 0.01401896 0.01278434 0.01445286]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
[0.02171887 0.01133982 0.02781667 0.01768976 0.04465854 0.0425582
 0.04354729 0.04740938 0.04232882 0.06090862]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
[0.01743029 0.01186956 0.0208675  0.01444567 0.02741407 0.02486467
 0.02627465 0.02917963 0.02619879 0.03554576]
(3760,)

    SELECT 
        -- [Met

In [372]:
x2

0.5956696556387302

In [373]:
toOptimize(x2)

[0.0185428  0.01173214 0.02267019 0.01528723 0.03188746 0.02945456
 0.03075535 0.03390862 0.03038309 0.04212515]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)


0.0

In [374]:
y = calculateVaRPLS (x2)
print(y)

[0.0185428  0.01173214 0.02267019 0.01528723 0.03188746 0.02945456
 0.03075535 0.03390862 0.03038309 0.04212515]
(3760,)

    SELECT 
        -- [Methodologie]
        --,[NombreObservation]
        --,[Percentile],
        [Rang]
        ,[Poids]
    FROM [rsq_model].[MesureRisqueMarche].[V_PonderationHarrellDavis]
    where Methodologie = '{}' and  [Percentile] = {} and 
    NombreObservation =  {} 
    
(3760,) (3760,)
0.13691006787276627


In [375]:
getVectorPLS(x2)

[0.0185428  0.01173214 0.02267019 0.01528723 0.03188746 0.02945456
 0.03075535 0.03390862 0.03038309 0.04212515]
(3760,)


array([ 0.0185428 ,  0.01173214,  0.02267019, ..., -0.00132276,
        0.0066984 ,  0.01261098])